# Convolutional Neural Network

## Cat and dog image classification model

### Import the libraries

In [3]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

## Data Preprocessing

### Preprocessing the training set

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,      # Feature scaling. Each pixel is between 0 and 255, we convert them to be between 0 and 1
    shear_range = 0.2,     # Shear mapping random skews the image.
    zoom_range = 0.2,      # Range at which it will randomly zoom into a picture
    horizontal_flip = True # Randomly flipping half of the images horizontally. Relevant when there are no assumptions ofm horizontal assymetry. (real-world photos)
)

training_set = train_datagen.flow_from_directory(
    '../data/training_set',
    target_size = (64, 64), # Size the image will be converted to (64px by 64px)
    batch_size = 32,        # Number of images in each batch (32)
    class_mode = 'binary'   # Classification type, binary or categorical, in this case it is cat and dog (binary)
)

Found 8000 images belonging to 2 classes.


### Preprocessing the test set

In [6]:
test_datagen = ImageDataGenerator(
    rescale = 1./255 # Feature scaling. Each pixel is between 0 and 255, we convert them to be between 0 and 1
) 

test_set = test_datagen.flow_from_directory(
    '../data/test_set',
    target_size = (64, 64), # Size the image will be converted to (64px by 64px)
    batch_size = 32,        # Number of images in each batch (32)
    class_mode = 'binary'   # Classification type, binary or categorical, in this case it is cat and dog (binary)
)

Found 2000 images belonging to 2 classes.


## Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential() # Stating the model as a sequence of layers

### Step 1 - Convolution

In [8]:
cnn.add(                          # .add is adding a new layer to the model
    tf.keras.layers.Conv2D(       # layers module. Conv2D class
        filters = 32,             # Number of feature detectors you want to apply (3)
        kernel_size = 3,          # Size of the feature detector. (3x3 in this case)
        activation = 'relu',      # Activation function. Choosing ReLU
        input_shape = [64, 64, 3] # Stating input shape (64px by 64px, 3 layers being RGB) (1 layer would be black and white)
    )
)

### Step 2 - Pooling

In [9]:
cnn.add(
    tf.keras.layers.MaxPool2D( # Layers module. MaxPool2D class.
        pool_size = 2,         # Width and height of square which scans the feature map (2)
        strides = 2            # Distance which the square moves. Equal to pool size so same place isn't scanned twice.
    )
)

### Adding a second convolutional layer

In [10]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters = 32,       # As above
        kernel_size = 3,    # As above
        activation = 'relu' # As above. Only need to state input_shape once on the first layer.
    )
)

cnn.add(
    tf.keras.layers.MaxPool2D(
        pool_size = 2,      # As above
        strides = 2         # As above
    )
)

### Step 3 - Flattening

In [11]:
cnn.add(
    tf.keras.layers.Flatten() # Flattens into a 1D vector.
)

### Step 4 - Full Connection

In [12]:
cnn.add(
    tf.keras.layers.Dense(  # Dense class.
        units = 128,        # Number of hidden neurons you want on the layer.
        activation = 'relu' # Activation function. Chosen ReLU.
    )
)

### Step 5 - Output Layer

In [13]:
cnn.add(
    tf.keras.layers.Dense(     # Dense class.
        units = 1,             # Number of neurons in output layer for our case is 1. Binary classification, need to know whether cat or dog.
        activation = 'sigmoid' # Sigmoid is for binary classification. Softmax is for categorical classification.
    )
)

## Training the CNN

### Compiling the CNN

In [14]:
cnn.compile(
    optimizer = 'adam',           # Adam optimiser for stochastic gradient descent to adjust the weights.
    loss = 'binary_crossentropy', # Binary cross entropy loss for binary classification.
    metrics = ['accuracy']        # Accuracy metrics because it's the most relevent way to check performance of classification model
)

### Training the CNN on the training set and evaluating it on the test set

In [15]:
cnn.fit(
    x = training_set,           # Specifying the traning set
    validation_data = test_set, # Specifying the test set
    epochs = 25                 # Number of epochs.
)

Epoch 1/25
250/250 [==============================] - 130s 521ms/step - loss: 0.6590 - accuracy: 0.6059 - val_loss: 0.5827 - val_accuracy: 0.7110
Epoch 2/25
250/250 [==============================] - 57s 230ms/step - loss: 0.5967 - accuracy: 0.6860 - val_loss: 0.6025 - val_accuracy: 0.6745
Epoch 3/25
250/250 [==============================] - 59s 235ms/step - loss: 0.5523 - accuracy: 0.7201 - val_loss: 0.5637 - val_accuracy: 0.7225
Epoch 4/25
250/250 [==============================] - 59s 236ms/step - loss: 0.5231 - accuracy: 0.7400 - val_loss: 0.5215 - val_accuracy: 0.7460
Epoch 5/25
250/250 [==============================] - 59s 235ms/step - loss: 0.5108 - accuracy: 0.7473 - val_loss: 0.4912 - val_accuracy: 0.7630
Epoch 6/25
250/250 [==============================] - 58s 232ms/step - loss: 0.4870 - accuracy: 0.7591 - val_loss: 0.5150 - val_accuracy: 0.7675
Epoch 7/25
250/250 [==============================] - 58s 233ms/step - loss: 0.4653 - accuracy: 0.7735 - val_loss: 0.4903 - val_a

## Making a prediction

In [35]:
test_image = image.load_img(
    '../data/single_prediction/sacha.jpg',
    target_size = (64, 64)                        # Specifying image size wanted to convert to as before.
)

test_image = image.img_to_array(test_image) # Converts into numpy array which is what is expected by next function.

test_image = np.expand_dims( # We trained on batches of images. We have an extra dimension of the batch. Single image needs to be as a batch still.
    test_image, # Image converted into a numpy array
    axis = 0    # Add extra dimension, converting image to a batch with only one image in. We want the first [0] dimension
)

result = cnn.predict(test_image) # Predict the preprocessed image. result = [[1.]]

training_set.class_indices # Get class indices from training set. {'cats': 0, 'dogs': 1}

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [36]:
print(prediction)

dog


In [19]:
cnn.save('../models/cnn/cats_and_dogs') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/cnn/cats_and_dogs/assets


In [32]:
# It can be used to reconstruct the model identically.
reconstructed_model = tf.keras.models.load_model("../models/cnn/cats_and_dogs")

# Let's check:
outcome = np.testing.assert_allclose(
    cnn.predict(test_image),
    reconstructed_model.predict(test_image)
)

# Assertion error raised if records don't match. No errors, model loaded successfully.